In [1]:
import pandas as pd
import psycopg2 as pg2

### Collect data and check statistics

In [2]:
data = pd.read_csv('./data/Airbnb_Open_Data.csv')

C:\Users\chide\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.shape

(102599, 26)

In [4]:
data.head()

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


In [5]:
data.count()

id                                102599
NAME                              102349
host id                           102599
host_identity_verified            102310
host name                         102193
neighbourhood group               102570
neighbourhood                     102583
lat                               102591
long                              102591
country                           102067
country code                      102468
instant_bookable                  102494
cancellation_policy               102523
room type                         102599
Construction year                 102385
price                             102352
service fee                       102326
minimum nights                    102190
number of reviews                 102416
last review                        86706
reviews per month                  86720
review rate number                102273
calculated host listings count    102280
availability 365                  102151
house_rules     

In [6]:
data.dtypes

id                                  int64
NAME                               object
host id                             int64
host_identity_verified             object
host name                          object
neighbourhood group                object
neighbourhood                      object
lat                               float64
long                              float64
country                            object
country code                       object
instant_bookable                   object
cancellation_policy                object
room type                          object
Construction year                 float64
price                              object
service fee                        object
minimum nights                    float64
number of reviews                 float64
last review                        object
reviews per month                 float64
review rate number                float64
calculated host listings count    float64
availability 365                  

In [7]:
data[['minimum nights',
      'number of reviews',
      'review rate number',
      'calculated host listings count', 
      'availability 365'
     ]].describe()

,minimum nights,number of reviews,review rate number,calculated host listings count,availability 365
count,102190.000000,102416.000000,102273.000000,102280.000000,102151.000000
mean,8.135845,27.483743,3.279106,7.936605,141.133254
std,30.553781,49.508954,1.284657,32.218780,135.435024
min,-1223.000000,0.000000,1.000000,1.000000,-10.000000
25%,2.000000,1.000000,2.000000,1.000000,3.000000
50%,3.000000,7.000000,3.000000,1.000000,96.000000
75%,5.000000,30.000000,4.000000,2.000000,269.000000
max,5645.000000,1024.000000,5.000000,332.000000,3677.000000


### Handle Missing and Null Values

In [8]:
#Check and drop Columns with over 86720 non Null values
df = data.dropna(axis=1, thresh=86720)
df

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,room type,Construction year,price,service fee,minimum nights,number of reviews,reviews per month,review rate number,calculated host listings count,availability 365
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,Private room,2020.0,$966,$193,10.0,9.0,0.21,4.0,6.0,286.0
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,Entire home/apt,2007.0,$142,$28,30.0,45.0,0.38,4.0,2.0,228.0
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,Private room,2005.0,$620,$124,3.0,0.0,NaN,5.0,1.0,352.0
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,Entire home/apt,2005.0,$368,$74,30.0,270.0,4.64,4.0,1.0,322.0
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,Entire home/apt,2009.0,$204,$41,10.0,9.0,0.10,3.0,1.0,289.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102594,6092437,Spare room in Williamsburg,12312296767,verified,Krik,Brooklyn,Williamsburg,40.70862,-73.94651,United States,...,Private room,2003.0,$844,$169,1.0,0.0,NaN,3.0,1.0,227.0
102595,6092990,Best Location near Columbia U,77864383453,unconfirmed,Mifan,Manhattan,Morningside Heights,40.80460,-73.96545,United States,...,Private room,2016.0,$837,$167,1.0,1.0,0.02,2.0,2.0,395.0
102596,6093542,"Comfy, bright room in Brooklyn",69050334417,unconfirmed,Megan,Brooklyn,Park Slope,40.67505,-73.98045,United States,...,Private room,2009.0,$988,$198,3.0,0.0,NaN,5.0,1.0,342.0
102597,6094094,Big Studio-One Stop from Midtown,11160591270,unconfirmed,Christopher,Queens,Long Island City,40.74989,-73.93777,United States,...,Entire home/apt,2015.0,$546,$109,2.0,5.0,0.10,3.0,1.0,386.0


In [9]:
#check datatypes and count of columns remaining
df.dtypes, df.count()

(id                                  int64
 NAME                               object
 host id                             int64
 host_identity_verified             object
 host name                          object
 neighbourhood group                object
 neighbourhood                      object
 lat                               float64
 long                              float64
 country                            object
 country code                       object
 instant_bookable                   object
 cancellation_policy                object
 room type                          object
 Construction year                 float64
 price                              object
 service fee                        object
 minimum nights                    float64
 number of reviews                 float64
 reviews per month                 float64
 review rate number                float64
 calculated host listings count    float64
 availability 365                  float64
 dtype: obj

In [10]:
# Fill some string objects with random words
df = df.fillna({
                 'NAME': 'No name',
                 'host_identity_verified': 'unconfirmed',
                 'host name': 'No name',
                 'neighbourhood group': 'No Neighbour',
                 'neighbourhood': 'Didnt Specify',
                 'instant_bookable': 'False',
                 'cancellation_policy': 'Any'})

In [11]:
# id, host-id and room type has no null values.
# Populate columns of float64 with 0.0
df1 = df[['Construction year','minimum nights',
         'number of reviews', 'reviews per month',
         'review rate number', 'calculated host listings count', 
         'availability 365']].fillna(0.0)

In [48]:
# Populate long, lat, country, country code with values after
df2 = df[['lat', 'long', 'country', 'country code']].fillna(method='bfill')

In [49]:
# fill price and service fee with $0
df3 = df[['price', 'service fee']].fillna('$0')

In [50]:
# Combine all filled columns to one dataframe
df_filled = pd.concat([df[['id','NAME', 'host id',
                 'host_identity_verified',
                 'host name',
                 'neighbourhood group',
                 'neighbourhood',
                 'instant_bookable',
                 'cancellation_policy', 'room type']], df1, df2, df3], axis=1)
df_filled.count()

id                                102599
NAME                              102599
host id                           102599
host_identity_verified            102599
host name                         102599
neighbourhood group               102599
neighbourhood                     102599
instant_bookable                  102599
cancellation_policy               102599
room type                         102599
Construction year                 102599
minimum nights                    102599
number of reviews                 102599
reviews per month                 102599
review rate number                102599
calculated host listings count    102599
availability 365                  102599
lat                               102599
long                              102599
country                           102599
country code                      102599
price                             102599
service fee                       102599
dtype: int64

### Handle Duplicates Values

In [51]:
# Check for duplicate values in dataframe
df_filled.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
102594     True
102595     True
102596     True
102597     True
102598     True
Length: 102599, dtype: bool

In [52]:
# Check for duplicates in id
df_filled.duplicated('id')

0         False
1         False
2         False
3         False
4         False
          ...  
102594     True
102595     True
102596     True
102597     True
102598     True
Length: 102599, dtype: bool

In [53]:
# Remove duplicates in id
df_no_dups = df_filled.drop_duplicates('id')
df_no_dups.shape

(102058, 23)

### Transform and Load to DB

In [54]:
# Change the datatype of prices and service fee 
# Remove $ and "," symbol
df4 = df_no_dups[['price', 'service fee']].replace("[$, ,]", "", regex=True)

In [55]:
# Convert the columns to int datatype
df4 = df4.astype('int64')

In [56]:
# Combine with other columns 
df_new = pd.concat([df_no_dups.iloc[:, 0:21], df4], axis=1)

In [57]:
df_new.head(3)

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,instant_bookable,cancellation_policy,room type,...,reviews per month,review rate number,calculated host listings count,availability 365,lat,long,country,country code,price,service fee
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,False,strict,Private room,...,0.21,4.0,6.0,286.0,40.64749,-73.97237,United States,US,966,193
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,False,moderate,Entire home/apt,...,0.38,4.0,2.0,228.0,40.75362,-73.98377,United States,US,142,28
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,unconfirmed,Elise,Manhattan,Harlem,True,flexible,Private room,...,0.00,5.0,1.0,352.0,40.80902,-73.94190,United States,US,620,124


In [58]:
# Rename some of the columns
df_renamed = df_new.rename(columns={
                           'NAME': 'airbnb_name',
                           'host id': 'host_id',
                           'host name': 'host_name',
                           'neighbourhood group': 'neighbourhood_group',
                           'room type': 'room_type',
                            'Construction year': 'construction_year',
                            'number of reviews': 'number_of_reviews',
                            'minimum nights': 'minimum_nights',
                            'country code': 'country_code',
                           'reviews per month': 'reviews_per_month',
                            'review rate number': 'review_rate_number',
                            'calculated host listings count': 'calculated_host_listings_count',
                            'availability 365': 'availability_365',
                            'service fee': 'service_fee'})

In [72]:
df_renamed.iloc[0]

id                                                           1001254
airbnb_name                       Clean & quiet apt home by the park
host_id                                                  80014485718
host_identity_verified                                   unconfirmed
host_name                                                   Madaline
neighbourhood_group                                         Brooklyn
neighbourhood                                             Kensington
instant_bookable                                               False
cancellation_policy                                           strict
room_type                                               Private room
construction_year                                             2020.0
minimum_nights                                                  10.0
number_of_reviews                                                9.0
reviews_per_month                                               0.21
review_rate_number                

In [ ]:
# # Change data type of some columns
# df_complete = df_renamed.astype({'airbnb_name': 'object'})

In [44]:
# Export data to PostgreSQL
# Connect to database 
conn = pg2.connect(database='postgres', user='postgres', password='****')

In [45]:
# Establish connection and start cursor to query
cur = conn.cursor()

In [65]:
# Create a new table called 
query = """
             CREATE TABLE airbnb (
                 id                                  varchar(50) PRIMARY KEY
                ,airbnb_name                         varchar(100)
                ,host_id                             varchar(50)
                ,host_identity_verified              varchar(20)
                ,host_name                           varchar(50)
                ,neighbourhood_group                 varchar(50)
                ,neighbourhood                       varchar(50)
                ,instant_bookable                    boolean
                ,cancellation_policy                 varchar(20)
                ,room_type                           varchar(50)
                ,construction_year                   date
                ,minimum_nights                      real
                ,number_of_reviews                   real
                ,reviews_per_month                   real
                ,review_rate_number                  real
                ,calculated_host_listings_count      real
                ,availability_365                    real
                ,lat                                 numeric
                ,long                                numeric
                ,country                             varchar(50)
                ,country_code                        varchar(10)
                ,price                               smallint
                ,service_fee                         smallint
             
             );
           
           """

In [67]:
cur.execute(query)

In [70]:
query1 = f"INSERT INTO airbnb VALUES({df_renamed.iloc[0]})"

In [71]:
cur.execute(query1)

SyntaxError: syntax error at or near "1001254"
LINE 1: ...                                                     1001254
                                                                ^
